# Word games assignment

You will be making a hangman game.
The words used for guessing will be taken from a corpus file full of sentences taken from Wikipedia.
10 marks are for neatness.

Below is a cell containing all the imports you will need in this assignment.
Run it and do not write another `import` anywhere else.

In [3]:
import re
import nltk
import random
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\dim12\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

Good luck!

## 1) Making a word list

In order to make a word guessing game, we need to have a list of words to guess.
You will extract these words from the corpus file provided with this assignment.

1.1) The corpus consists of one sentence per line.
Read these sentences from the file and put them in a list called `sentences`.
Print the length of this list, which should be 2239.

[5 marks]

In [5]:
file_path = 'corpus.txt'  
with open('corpus.txt', 'r', encoding='utf-8') as file:
    sentences = file.readlines()
sentences = [sentence.strip() for sentence in sentences]
print(len(sentences))   # Should print 2239

2239


1.2) Extract all the words from `sentences` and put them all into one list called `words`.
Print the length of this list, which should be 32469.

[5 marks]

In [7]:
from nltk.tokenize import word_tokenize
words = []
for sentence in sentences:
    words_in_sentence = word_tokenize(sentence)
    words.extend(words_in_sentence)

print(len(words)) 

32469


1.3) Uncommon words make word guessing games too difficult, so we shall focus on the frequent words only.
Get the frequencies of all the words in `words` and create a list called `freq_words` of all the words that occur 5 times or more.
Print the length of `freq_words`, which should be 796.

[5 marks]

In [9]:
word_counts = nltk.FreqDist(words)

freq_words = [word for word, count in word_counts.items() if count >= 5]
print(len(freq_words))

796


1.4) Finally, create a list called `game_words` and fill it with all the words in `freq_words` that consist of only lowercase letters of the English alphabet and that are between 3 and 10 characters long.
Use a regular expression to check both of these conditions at once.
Print the length of `game_words`, which should be 541.

These will be the words we use in our games.

[5 marks]

In [11]:

game_words = [word for word in freq_words if re.match(r'^[a-z]{3,10}$', word)]
#print(game_words[:100])
print(len(game_words))  # This should print 541

541


## 2) Hangman game

In a game of hangman, the computer chooses a random word and displays the word using blanks instead of letters, for example '\*\*\*\*'.
The user then has to guess a letter that is in the word, and is shown all the letters that have not been tried yet.
When a letter is guessed correctly, the blanks in the word that were hiding the guesssed letter are replaced with the letter, for example '\*ee\*'.
Whether the guessed letter is in the word or not, the letter will still be removed from the list of letters that have not been tried yet.
We will not be drawing an actual picture of a man being hanged as that takes away from the actual programming task.

2.1) Start by writing a function called `get_guess` that has a parameter called `valid_letters` (a set of letters), and that **repeatedly** asks the user to input a letter until the inputted letter is both a single letter and in `valid_letters`.
Every time the inputted letter is not a single letter, print that it is not a letter.
Every time the inputted letter is not in `valid_letters`, print that it is not an available letter.
The function should return the inputted letter if it meets both criteria.

Some test code is provided for you to test your function.

[10 marks]

In [13]:
def get_guess(valid_letters):
    while True:
        guess = input("Please enter a letter: ").lower()
        if len(guess) != 1 or not guess.isalpha():
            print("That is not a letter. Please try again.")
        elif guess not in valid_letters:
            print("That is not an available letter. Please try again.")
        else:
            return guess

In [17]:
# Don't change!

print('Entering \'x\' should say that it is not an available letter.')
print('Entering \'aa\' should say that it is not a single letter.')
print('Entering \'a\' should print \'a\' and stop asking for an input.')
print(get_guess({'a', 'b', 'c'}))

Entering 'x' should say that it is not an available letter.
Entering 'aa' should say that it is not a single letter.
Entering 'a' should print 'a' and stop asking for an input.


Please enter a letter:  x


That is not an available letter. Please try again.


Please enter a letter:  aa


That is not a letter. Please try again.


Please enter a letter:  a


a


2.2) Write a function called `sort` that takes a set of letters and returns all the letters in the set as single string with all the letters in sorted order.

[5 marks]

In [19]:
def sort(letter_set): #set => list
    sorted_letters = sorted(letter_set) #made to single string
    return ''.join(sorted_letters)

In [21]:
# Don't change!

print('Should print "abcd":', sort({'a', 'b', 'c', 'd'}))

Should print "abcd": abcd


2.3) Make a function called `cover_letters` that takes a word and a set of unguessed letters and returns the word with all the unguessed letters replaced by '\*'.

Some test code is provided to help you check your function.

[5 marks]

In [23]:
def cover_letters(word, unguessed_letters):
    covered_word = ''.join(letter if letter not in unguessed_letters else '*' for letter in word)
    return covered_word

In [25]:
# Don't change!

print('Should print "beer":', cover_letters('beer', set('')))
print('Should print "beer":', cover_letters('beer', set('acdfghijklmnopqstuvwxyz')))
print('Should print "*eer":', cover_letters('beer', set('b')))
print('Should print "*ee*":', cover_letters('beer', set('abcdfghijklmnopqrstuvwxyz')))
print('Should print "****":', cover_letters('beer', set('abcdefghijklmnopqrstuvwxyz')))

Should print "beer": beer
Should print "beer": beer
Should print "*eer": *eer
Should print "*ee*": *ee*
Should print "****": ****


2.4) Next, write a class called `LettersManager` with the following:

* Has an initialiser that takes `target_word` as a parameter which is the target word to guess.
* Has two instance variables called `self.unused_correct_letters` and `self.unused_letters`.
    `self.unused_correct_letters` is a set containing all the correct letters that were not guessed yet and should be initialised to the letters in `target_word`.
    `self.unused_letters` is a set containing all the letters of the alphabet there weren't attempted yet (neither correctly nor incorrectly) and should be initialised to the letters of the alphabet (abcdefghijklmnopqrstuvwxyz).
* Has a method called `get_unused_correct_letters`.
* Has a method called `get_unused_letters`.
* Has a method called `is_correct` that takes a letter and returns if the letter is a correct one.
* Has a method called `is_guessed` that returns if the whole word has been guessed.
* Has a method called `remove` that takes a letter and removes it from both `self.unused_correct_letters` and `self.unused_letters`.

Some test code is provided to help you check your code.

[25 marks]

In [27]:
class LettersManager:
    def __init__(self, target_word):
        self.target_word = target_word
        self.unused_correct_letters = set(target_word)
        self.unused_letters = set('abcdefghijklmnopqrstuvwxyz')
        
    def get_unused_correct_letters(self):
        return self.unused_correct_letters
        
    def get_unused_letters(self):
        return self.unused_letters

    def is_correct(self, letter):
        return letter in self.unused_correct_letters

    def is_guessed(self):
        return len(self.unused_correct_letters) == 0

    def remove(self, letter):
        self.unused_correct_letters.discard(letter)
        self.unused_letters.discard(letter)

In [29]:
# Don't change!

mngr = LettersManager('beer')
print('Should be "ber":', sort(mngr.get_unused_correct_letters()))
print('Should be all the letters of the alphabet:', sort(mngr.get_unused_letters()))
print('Should be False:', mngr.is_correct('x'))
print('Should be True:', mngr.is_correct('b'))
print('Should be False:', mngr.is_guessed())
print()

mngr.remove('b')
print('Should be "er":', sort(mngr.get_unused_correct_letters()))
print('Should be all the letters of the alphabet except for b:', sort(mngr.get_unused_letters()))
print('Should be False:', mngr.is_correct('x'))
print('Should be False:', mngr.is_correct('b'))
print('Should be True:', mngr.is_correct('e'))
print('Should be False:', mngr.is_guessed())
print()

mngr.remove('x')
print('Should be "er":', sort(mngr.get_unused_correct_letters()))
print('Should be all the letters of the alphabet except for b and x:', sort(mngr.get_unused_letters()))
print('Should be False:', mngr.is_correct('x'))
print('Should be False:', mngr.is_correct('b'))
print('Should be True:', mngr.is_correct('e'))
print('Should be False:', mngr.is_guessed())
print()

mngr.remove('e')
print('Should be "r":', sort(mngr.get_unused_correct_letters()))
print('Should be all the letters of the alphabet except for b, x, and e:', sort(mngr.get_unused_letters()))
print('Should be False:', mngr.is_correct('x'))
print('Should be False:', mngr.is_correct('b'))
print('Should be False:', mngr.is_correct('e'))
print('Should be False:', mngr.is_guessed())
print()

mngr.remove('r')
print('Should be "":', sort(mngr.get_unused_correct_letters()))
print('Should be all the letters of the alphabet except for b, x, e, and r:', sort(mngr.get_unused_letters()))
print('Should be True:', mngr.is_guessed())

Should be "ber": ber
Should be all the letters of the alphabet: abcdefghijklmnopqrstuvwxyz
Should be False: False
Should be True: True
Should be False: False

Should be "er": er
Should be all the letters of the alphabet except for b: acdefghijklmnopqrstuvwxyz
Should be False: False
Should be False: False
Should be True: True
Should be False: False

Should be "er": er
Should be all the letters of the alphabet except for b and x: acdefghijklmnopqrstuvwyz
Should be False: False
Should be False: False
Should be True: True
Should be False: False

Should be "r": r
Should be all the letters of the alphabet except for b, x, and e: acdfghijklmnopqrstuvwyz
Should be False: False
Should be False: False
Should be False: False
Should be False: False

Should be "": 
Should be all the letters of the alphabet except for b, x, e, and r: acdfghijklmnopqstuvwyz
Should be True: True


2.5) Now comes the main code of the game.
Write code that does the following:

* Randomly pick a word from `game_words` and put it in a variable called `target_word`.
* Create a `LettersManager` object and put it in a variable called `letters_mngr`
* Repeatedly do the following:
    * Print the number of **failed** letter guesses that have been made (increases every time an inputted letter was not correct).
    * Print the letters that the user hasn't used yet.
    * Print the covered version of the target word (covering the unguessed letters).
    * Call `get_guess` and put the returned letter in a variable called `letter`.
    * Remove `letter` from `letters_mngr`.
    * Check if the word has been guessed (`letters_mngr.is_guessed`) and end the loop if so.

Make sure that a nice message is shown when the word is completely guessed and that the entire output looks neat.

[25 marks]

In [31]:
target_word = random.choice(game_words)
print(f"The randomly selected target word is: {target_word}")


def get_guess(valid_letters):
    while True:
        guess = input("Please enter a letter: ").lower()
        if len(guess) != 1 or not guess.isalpha():
            print("That is not a letter. Please try again.")
        elif guess not in valid_letters:
            print("That is not an available letter. Please try again.")
        else:
            return guess

# Function to cover letters in the word
def cover_letters(word, unguessed_letters):
    return ''.join(letter if letter not in unguessed_letters else '*' for letter in word)

# LettersManager class
class LettersManager:
    def __init__(self, target_word):
        self.target_word = target_word
        self.unused_correct_letters = set(target_word)
        self.unused_letters = set('abcdefghijklmnopqrstuvwxyz')

    def get_unused_correct_letters(self):
        return self.unused_correct_letters

    def get_unused_letters(self):
        return self.unused_letters

    def is_correct(self, letter):
        return letter in self.unused_correct_letters

    def is_guessed(self):
        return len(self.unused_correct_letters) == 0

    def remove(self, letter):
        self.unused_correct_letters.discard(letter)
        self.unused_letters.discard(letter)

# Main game code
if __name__ == "__main__":
    target_word = random.choice(game_words)
    letters_mngr = LettersManager(target_word)

    failed_guesses = 0

    while True:
        print(f"Failed guesses: {failed_guesses}")
        print("Unused letters:", ''.join(sorted(letters_mngr.get_unused_letters())))
        covered_word = cover_letters(target_word, letters_mngr.get_unused_correct_letters())
        print("Current word:", covered_word)

        letter = get_guess(letters_mngr.get_unused_letters())
        letters_mngr.remove(letter)

        if not letters_mngr.is_correct(letter):
            failed_guesses += 1

        if letters_mngr.is_guessed():
            print(f"Congratulations! You've guessed the word: {target_word}")
            break

The randomly selected target word is: between
Failed guesses: 0
Unused letters: abcdefghijklmnopqrstuvwxyz
Current word: ******


Please enter a letter:  a


Failed guesses: 1
Unused letters: bcdefghijklmnopqrstuvwxyz
Current word: ******


Please enter a letter:  e


Failed guesses: 2
Unused letters: bcdfghijklmnopqrstuvwxyz
Current word: ******


...

Failed guesses: 13
Unused letters: cfhjknqsvwxyz
Current word: *oug*t


Please enter a letter:  s


Failed guesses: 14
Unused letters: cfhjknqvwxyz
Current word: soug*t


Please enter a letter:  h


Congratulations! You've guessed the word: sought
